# Orange brix prediction

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive//MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing/"
# Master_homework/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Master_homework/Advanced_programming_for_scientific_computing


## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from enum import Enum
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error

import os

In [ ]:
!pip install lightgbm
!pip install xgboost

In [ ]:
plt.style.use("fivethirtyeight")

In [ ]:
GENERATE_PLOTS: bool = False

## Create Classes

In [ ]:
class Utilities:
    def __init__(self):
        pass

    def get_categorical_numeric_cols(self, dataframe: pd.DataFrame):
        if len(dataframe) == 0:
            raise Exception("No records found!")

        numeric_cols = []
        categorical_cols = []

        for col in dataframe.columns:
            if isinstance(dataframe[col][0], bool) or not isinstance(
                dataframe[col][0], (int, float, np.int8, np.int16, np.int32, np.int64, np.float16, np.float32, np.float64)
            ):
                categorical_cols.append(col)
            else:
                numeric_cols.append(col)

        return (categorical_cols, numeric_cols)

In [ ]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    RobustScaler,
    StandardScaler,
)


class ScalerType(Enum):
    Raw_ = None
    MaxAbsScaler_ = MaxAbsScaler
    MinMaxScaler_ = MinMaxScaler
    Normalizer_ = Normalizer
    RobustScaler_ = RobustScaler
    StandardScaler_ = StandardScaler

In [ ]:
class Dataset(Utilities):
    def __init__(
        self,
        name: str,
        dataframe: pd.DataFrame,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):
        self.name: str = name
        self.dataframe: pd.DataFrame = dataframe
        self.categorical_cols: list[str] = []
        self.numeric_cols: list[str] = []
        self.scaler_type: ScalerType = scaler_type

        self.categorical_cols, self.numeric_cols = self.get_categorical_numeric_cols(
            self.dataframe
        )

    def get_name(self) -> str:
        return self.name

    def get_dataframe(self) -> pd.DataFrame:
        return self.dataframe

    def detect_categorical_numeric_cols(self):
        if len(self.dataframe) == 0:
            raise Exception("No records found!")

        self.numeric_cols = []
        self.categorical_cols = []

        for col in self.dataframe.columns:
            try:
                float(self.dataframe[col][0])
                self.numeric_cols.append(col)
            except:
                self.categorical_cols.append(col)

    def get_numeric_cols(self) -> list[str]:
        return self.numeric_cols

    def get_numeric_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.numeric_cols]

    def get_categorical_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.categorical_cols]

    def get_scaler_type(self) -> ScalerType:
        return self.scaler_type

    def concat_dataframe(self, additional_dataframe: pd.DataFrame):
        if len(additional_dataframe) == len(self.dataframe):
            self.dataframe = pd.concat([self.dataframe, additional_dataframe], axis=1)
            self.detect_categorical_numeric_cols()

    def get_one_hot_vectorized_dataset(
        self, categorical_col: str, print_name: bool = True
    ):
        if categorical_col in self.categorical_cols:
            dummies_dataframe = pd.get_dummies(self.dataframe[categorical_col])
            dummies_dataframe = pd.concat([dummies_dataframe, self.dataframe], axis=1)
            new_dataset_name = self.name + "_ohv_" + categorical_col
            if print_name:
                print(new_dataset_name)
            return Dataset(
                name=new_dataset_name,
                dataframe=dummies_dataframe,
                scaler_type=self.scaler_type,
            )
        elif categorical_col in self.numeric_cols:
            print("Records for input column name are not categorical!")
        else:
            print("Input column name does not exists!")

In [ ]:
class ModelType(Enum):
    LINEAR_REGRESSION: dict = {}
    SVR: dict = {
        "kernel": "rbf", # imp | linear, poly, rbf
        "degree": 3,
        "gamma": "scale",
        "coef0": 0,
        "tol": 1e-3,
        "C": 1.0, # imp tune from 0.1 - 2 step 0.1
        "epsilon": 0.1,
    }
    DECISION_TREE: dict = {
        "criterion": "squared_error",
        "splitter": "best",
        "max_depth": None, # imp tune 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": None, # int 2->5 {“sqrt”, “log2”}, default=None

        "random_state": 42,
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0,
    }
    RANDOM_FOREST: dict = {
        "n_estimators": 100, # int, 100-1000 (step 100)
        "criterion": "squared_error",
        "max_depth": None, # imp tune 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": 1.0, # int 2->5 {“sqrt”, “log2”}, default=None
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0.0,
    }
    LIGHT_GBM: dict = {
        "boosting_type": "gbdt",
        "num_leaves": 31,
        "max_depth": -1,
        "learning_rate": 0.1,
        "n_estimator": 100, # int, 100->1000 (step 100)
        "verbosity": -1,
    }
    XGB: dict = {
        "objective": "reg:linear",
        "n_estimators": 10, # int, 100-1000 (step 100)
        "seed": 123,
        "verbosity": 0,
    }


In [ ]:
class Result:
    def __init__(
        self,
        dataset_name: str,
        model_name: str,
        model_type: ModelType,
        scaler_type: str,
        r2: float,
        mse: float,
    ):
        self.result = [dataset_name, model_name, model_type, scaler_type, r2, mse]

    def get_result(self):
        return self.result

In [ ]:
class ResultFrame:
    def __init__(self):
        self.result_frame: pd.DataFrame = pd.DataFrame()
        self.count = 0

        self.start_up()

    def start_up(self):
        self.result_frame["dataset_name"] = []
        self.result_frame["model_name"] = []
        self.result_frame["model_type"] = []
        self.result_frame["scaler_type"] = []
        self.result_frame["r2"] = []
        self.result_frame["mse"] = []

    def add_result(self, new_result: Result):
        self.result_frame.loc[self.count] = new_result.result
        self.count += 1

    def display_result(self):
        self.result_frame.sort_values(["r2", "mse"], ascending=False, inplace=True)
        display(self.result_frame)

    def save_result(self, filename, overwrite=False):
        file_exists = os.path.isfile(os.path.join(os.getcwd(), filename))
        if file_exists and not overwrite:
            print("Error: CANNOT SAVE FILE.")
            print("A file with the same name already exists.")
            print(
                "Set overwrite to True to overwrite existed file or change file name!"
            )
        else:
            self.result_frame.to_excel(filename)

In [ ]:
class Model(Utilities):
    def __init__(
        self,
        model_type: ModelType,
        name: str,
        x_cols: list[str],
        y_cols: list[str],
        dataset: Dataset,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):

        if len(x_cols) == 0 or len(y_cols) == 0:
            raise Exception("Data columns are not specified!")

        self.model_type: ModelType = model_type
        self.name: str = name
        self.scaler_type: ScalerType = scaler_type
        self.scaler = self.get_scaler(self.scaler_type)  # could be None
        self.x_dataset: Dataset = None
        self.y_dataset: Dataset = None
        self.inner_model = None

        self.preprocess_data(x_cols, y_cols, dataset)

    # Extract data from x_cols and y_cols
    # Redetect categorical and numeric columns
    def preprocess_data(self, x_cols: list[str], y_cols: list[str], dataset: Dataset):
        x_dataset_new_name = dataset.get_name() + "_x"
        y_dataset_new_name = dataset.get_name() + "_x"
        scaler_type = dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=x_dataset_new_name,
            dataframe=dataset.get_dataframe()[x_cols],
            scaler_type=scaler_type,
        )
        self.y_dataset = Dataset(
            name=y_dataset_new_name,
            dataframe=dataset.get_dataframe()[y_cols],
            scaler_type=scaler_type,
        )

        self.process_x_dataset()

    # arrange [numeric_cols, categorical_cols]
    def process_x_dataset(self):
        new_name = self.x_dataset.get_name()
        numeric_dataframe = self.x_dataset.get_numeric_dataframe()
        categorical_dataframe = self.x_dataset.get_categorical_dataframe()
        scaler_type = self.x_dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=new_name,
            dataframe=pd.concat([numeric_dataframe, categorical_dataframe], axis=1),
            scaler_type=scaler_type,
        )

    def get_x_list_index_numeric_cols(self):
        return list(range(len(self.x_dataset.get_numeric_cols())))

    def fit_scaler_and_scale_dataframe(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            self.scaler.fit(X_numeric)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X

        return transformed_X

    def get_x_dataframe(self):
        return self.x_dataset.get_dataframe()

    def get_y_dataframe(self):
        return self.y_dataset.get_dataframe()

    def get_scaler(self, scaler_type: ScalerType):
        if scaler_type != ScalerType.Raw_:
            return scaler_type.value()

    def fit(self, X=None, Y=None):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())

        model_family = self.model_type.__str__().split(".")[-1]
        params = self.model_type.value

        Y = np.squeeze(Y)

        # need to scale
        transformed_X = self.fit_scaler_and_scale_dataframe(X)

        if model_family == "SVR":
            self.model = SVR(**params).fit(transformed_X, Y)
        elif model_family == "LINEAR_REGRESSION":
            self.model = LinearRegression(**params).fit(transformed_X, Y)
        elif model_family == "DECISION_TREE":
            self.model = DecisionTreeRegressor(**params).fit(transformed_X, Y)
        elif model_family == "RANDOM_FOREST":
            self.model = RandomForestRegressor(**params).fit(transformed_X, Y)
        elif model_family == "LIGHT_GBM":
            self.model = LGBMRegressor(**params).fit(transformed_X, Y)
        elif model_family == "XGB":
            self.model = XGBRegressor(**params).fit(transformed_X, Y)
        else:
            raise ValueError("Unknown model family!")

    def predict(self, X):
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X
        return self.model.predict(transformed_X)

    def evaluate(self, X=None, Y=None, method="LOOCV"):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())
        if method == "LOOCV":
            preds = [0] * len(Y)
            total = len(Y)
            kf = KFold(n_splits=total)
            kf.get_n_splits(Y)

        with tqdm(total=total) as pbar:
            for i, (train_index, valid_index) in enumerate(kf.split(X, Y)):
                X_train = X[train_index]
                Y_train = Y[train_index]
                X_valid = X[valid_index]
                Y_valid = Y[valid_index]
                self.fit(X_train, Y_train)

                Y_valid_pred = self.predict(X_valid)
                for j in range(len(valid_index)):
                    index = valid_index[j]
                    value = Y_valid_pred[j]
                    preds[index] = value
                pbar.update(1)

        r2 = r2_score(Y, preds)
        mse = mean_squared_error(Y, preds)

        # Compile result
        dataset_name = self.x_dataset.get_name()[:-2]
        return Result(
            dataset_name=dataset_name,
            model_name=self.name,
            model_type=self.model_type,
            scaler_type=self.scaler_type,
            r2=r2,
            mse=mse,
        )

    def save(self, filename: str):
        pass

    def load(self, filename: str):
        pass

    def set_params(self, params: dict):

        pass

## Create main variables

In [ ]:
list_dataset: dict[str, Dataset] = dict()
list_model: dict = dict()


## Create functions

In [ ]:
def list_all_dataset_names():
    return list_dataset.keys()

In [ ]:
def list_all_model_names():
    return list_model.keys()

In [ ]:
def plot_along_dataset(
    dataset: Dataset,
    nrow_subplot: int,
    ncol_subplot: int,
    figsize: tuple[int, int],
):
    numeric_df = dataset.get_dataframe()[dataset.numeric_cols]
    x = np.arange(len(numeric_df))
    figs, axes = plt.subplots(nrow_subplot, ncol_subplot, figsize=figsize)
    for i in range(nrow_subplot):
        for j in range(ncol_subplot):
            ax = axes[i][j]
            index = i * ncol_subplot + j
            ax.plot(x, numeric_df.iloc[:, index])
            ax.set_title(f"{numeric_df.columns[index]} along dataframe", size=15)
            ax.xaxis.set_tick_params(labelsize=12)
            ax.xaxis.set_ticks(np.arange(0, len(numeric_df), 10))
            ax.yaxis.set_tick_params(labelsize=12)

In [ ]:
def plot_hist_kde_box_all_columns(dataset: Dataset):

    number_of_numeric_col = len(dataset.numeric_cols)
    figs, axes = plt.subplots(
        number_of_numeric_col, 3, figsize=(30, number_of_numeric_col * 10)
    )

    numeric_dataframe = dataset.get_dataframe()[dataset.numeric_cols]

    for i in range(number_of_numeric_col):
        axes[i][0].set_title(f"{numeric_dataframe.columns[i]} histogram", size=20)
        axes[i][0].yaxis.set_tick_params(labelsize=15)
        axes[i][0].xaxis.set_tick_params(labelsize=15)
        sns.histplot(ax=axes[i][0], data=numeric_dataframe.iloc[:, i])

        axes[i][1].set_title(f"{numeric_dataframe.columns[i]} KDE", size=20)
        axes[i][1].xaxis.set_tick_params(labelsize=15)
        axes[i][1].yaxis.set_tick_params(labelsize=15)
        sns.kdeplot(ax=axes[i][1], data=numeric_dataframe.iloc[:, i])

        axes[i][2].set_title(f"{numeric_dataframe.columns[i]} Boxplot", size=20)
        axes[i][2].xaxis.set_tick_params(labelsize=15)
        axes[i][2].yaxis.set_tick_params(labelsize=15)
        sns.boxplot(ax=axes[i][2], data=numeric_dataframe.iloc[:, i])

In [ ]:
def plot_all_columns_group_by_a_categorical_col(dataset: Dataset, categorical_col: str):

    dataframe = dataset.get_dataframe()
    numeric_dataframe = dataframe[dataset.numeric_cols]
    number_of_numeric_col = len(dataset.numeric_cols)
    figs, axes = plt.subplots(
        number_of_numeric_col, 2, figsize=(20, number_of_numeric_col * 10)
    )
    groups = dataframe[categorical_col].unique()

    for i in range(number_of_numeric_col):
        axes[i][0].set_title(f"{numeric_dataframe.columns[i]} KDE", size=20)
        axes[i][0].xaxis.set_tick_params(labelsize=15)
        axes[i][0].yaxis.set_tick_params(labelsize=15)
        for group in groups:
            sns.kdeplot(
                ax=axes[i][0],
                data=numeric_dataframe[dataframe[categorical_col] == group].iloc[:, i],
                label=f"Type {group}",
            )
        axes[i][0].legend()

        axes[i][1].set_title(f"{numeric_dataframe.columns[i]} Boxplot", size=20)
        axes[i][1].xaxis.set_tick_params(labelsize=15)
        axes[i][1].yaxis.set_tick_params(labelsize=15)
        sns.boxplot(
            ax=axes[i][1],
            hue="type",
            y=numeric_dataframe.columns[i],
            data=dataframe,
        )

In [ ]:
def divide_dataframe_with_categorical_col(
    dataset: Dataset, categorical_col: str
) -> dict[str, pd.DataFrame]:
    dataframe = dataset.get_dataframe()
    groups = dataframe[categorical_col].unique()

    result = {}
    for group in groups:
        result[group] = dataframe[dataframe[categorical_col] == group]

    return result

In [ ]:
def calculate_p_values_between_partitions(dataframes, columns_to_compare):
    p_values = {}
    for key1, df1 in dataframes.items():
        for key2, df2 in dataframes.items():
            if key1 != key2:
                for col in columns_to_compare:
                    if col in df1 and col in df2:
                        t_stat, p_val = stats.ttest_ind(df1[col], df2[col])
                        p_values[f"{col}_{key1}-{col}_{key2}"] = p_val
    return p_values

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from typing import Optional, Callable


def summary_linear_model(x: list, y, dataset, transformer: Optional[Callable] = None):
    x_ = sm.add_constant(dataset[x])
    y_ = dataset[y]
    results = sm.OLS(y_, x_).fit()
    Intercept = results.params.iloc[0]
    Slope = results.params.iloc[1:]
    P_values = results.pvalues[1:]
    R_squared = results.rsquared
    MSE = mean_squared_error(y_, results.predict())
    print("Intercept\n", Intercept, end="\n\n")
    print("Slope\n", Slope, end="\n\n")
    print("P_values\n", P_values, end="\n\n")
    print("R_squared\n", R_squared, end="\n\n")
    print("MSE\n", MSE, end="\n\n")
    return {
        "Intercept": Intercept,
        "Slope": Slope,
        "P_values": P_values,
        "R_squared": R_squared,
        "MSE": MSE,
    }

In [ ]:
# intercept_a = model_no_scale["Slope"]["A"] + model_no_scale["Intercept"]
# intercept_b = model_no_scale["Slope"]["B"] + model_no_scale["Intercept"]
# intercept_c = model_no_scale["Slope"]["C"] + model_no_scale["Intercept"]

# x = np.arange(model_dataset["mass"].min(), model_dataset["mass"].max())
# ya = x * model_no_scale["Slope"]["mass"] + intercept_a
# yb = x * model_no_scale["Slope"]["mass"] + intercept_b
# yc = x * model_no_scale["Slope"]["mass"] + intercept_c
# plt.plot(x, ya, linewidth=2, label="type_A")
# plt.plot(x, yb, linewidth=2, label="type_B")
# plt.plot(x, yc, linewidth=2, label="type_C")
# plt.legend()

## Import data

In [ ]:
!ls

cropped			   result_eva_large_patch14_196.in22k_ft_in22k_in1k.xlsx
cropped.rar		   result_KNN_get_10_new.xlsx
data_v2			   result_LIGHT_GBM_get_10.xlsx
Dulieu			   result_LIGHT_GBM_get_mean.xlsx
Harris_detector.png	   result_LIGHT_GBM.xlsx
legacy			   result_resnet_new.xlsx
orange_data.xlsx	   result_vit_base_patch16_224.dino_best_54.xlsx
orange.ipynb		   result_vit_base_patch16_224.dino_num_leaves_exp.xlsx
orange_v2.ipynb		   result_vit_base_patch16_224.dino.xlsx
orange_v3.ipynb		   result_vit_large_patch14_dinov2.lvd142m.xlsx
Orange_vision.ipynb	   result_XGB_get_10_new.xlsx
Pham_Minh_Hieu_Tuan1	   result_XGB_prune.xlsx
Pham_Minh_Hieu_Tuan2	   result_XGB.xlsx
Pham_Minh_Hieu_Tuan3	   timm
result_all_models.xlsx	   TIMM.ipynb
result_best_0.547777.xlsx


In [ ]:
raw_df = pd.read_excel("orange_data.xlsx", sheet_name=0)

In [ ]:
raw_df.head(10)

,Name,p,C NaOH,Khối lượng(g),Đường kính ngang(mm),Đường kính dọc(mm),Độ đường (Brix %),TA (%),Brix:TA,BrmTA
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [ ]:
raw_df.columns

Index(['Name', 'p', 'C NaOH', 'Khối lượng(g)', 'Đường kính ngang(mm)',
       'Đường kính dọc(mm)', 'Độ đường (Brix %)', 'TA (%)', 'Brix:TA',
       'BrmTA'],
      dtype='object')

In [ ]:
raw_df.shape

(81, 10)

In [ ]:
new_column_names = {
    "Name": "name",
    "p": "v_naoh",
    "C NaOH": "c_naoh",
    "Khối lượng(g)": "mass",
    "Đường kính ngang(mm)": "h_diameter",
    "Đường kính dọc(mm)": "v_diameter",
    "Độ đường (Brix %)": "brix",
    "TA (%)": "ta",
    "Brix:TA": "brix_ta",
    "BrmTA": "brm_ta",
}

std_col_name_df = raw_df.rename(columns=new_column_names)

In [ ]:
std_col_name_df.head(10)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [ ]:
std_col_name_dataset = Dataset("std_col_name", std_col_name_df, ScalerType.Raw_)

In [ ]:
print(std_col_name_dataset.categorical_cols)
print(std_col_name_dataset.numeric_cols)
std_col_name_dataset.dataframe

['name']
['v_naoh', 'c_naoh', 'mass', 'h_diameter', 'v_diameter', 'brix', 'ta', 'brix_ta', 'brm_ta']


,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
...,...,...,...,...,...,...,...,...,...,...
76,C24,1.60,0.4016,177,225,216,12.5,4.115040,3.037638,8.384960
77,C25,1.60,0.4016,173,216,220,11.1,4.115040,2.697422,6.984960
78,C26,0.95,0.4016,126,195,197,12.8,2.443305,5.238806,10.356695
79,C28,1.70,0.4016,181,216,232,11.8,4.372230,2.698852,7.427770


In [ ]:
list_dataset["std_col_name"] = std_col_name_dataset

In [ ]:
list_dataset["std_col_name"].get_dataframe().head(10)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748


In [ ]:
if GENERATE_PLOTS:
    plot_along_dataset(list_dataset["std_col_name"], 3, 3, (12, 12))

#### From the shape of lines in graph "mass", "h_diameter", "v_diameter" and "brix", we can see the lines could be divided into 3 levels along dataset. We assume that there are 3 types of data in the dataset. We could double check the name / label of dataset.

In [ ]:
list_dataset["std_col_name"].get_dataframe().columns

Index(['name', 'v_naoh', 'c_naoh', 'mass', 'h_diameter', 'v_diameter', 'brix',
       'ta', 'brix_ta', 'brm_ta'],
      dtype='object')

In [ ]:
print(list_dataset["std_col_name"].get_dataframe()["name"].to_list())

['A1', 'A2', 'A3', 'A4', 'A5', 'A7', 'A8', 'A9', 'A10', 'A11', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16', 'B17', 'B18', 'B19', 'B20', 'B21', 'B22', 'B23', 'B24', 'B25', 'B26', 'B27', 'B28', 'B29', 'B30', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C8', 'C9', 'C10', 'C11', 'C12', 'C14', 'C15', 'C16', 'C17', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C28', 'C29']


We count the number of type to see whether the amount is equivalent to the shape of lines in the following graphs: "mass", "h_diameter", "v_diameter" and "brix"

In [ ]:
import re

data_type = []
for name in list_dataset["std_col_name"].get_dataframe()["name"].str.upper():
    data_type.append(re.findall("[A-Z]", name)[0])

data_type = pd.DataFrame(data_type, columns=["type"])
data_type["count"] = 1
data_type.groupby(["type"]).count()["count"]

type
A    26
B    30
C    25
Name: count, dtype: int64

In [ ]:
list_dataset["std_col_name"].concat_dataframe(data_type["type"])

In [ ]:
list_dataset["std_col_name"].get_dataframe().head(10)

,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
5,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286,A
6,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693,A
7,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110,A
8,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458,A
9,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748,A


## Analyze quantitative data

### Plots for all rows

In [ ]:
if GENERATE_PLOTS:
    plot_hist_kde_box_all_columns(list_dataset["std_col_name"])

In [ ]:
list_dataset["std_col_name"].get_dataframe().describe()

,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta
count,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,1.477778,0.406857,222.197531,241.777778,241.358025,11.169259,3.849815,3.024935,7.319445
std,0.297069,0.007092,50.582215,23.652167,22.587225,1.362702,0.774895,0.786520,1.587514
min,0.800000,0.400000,126.000000,195.000000,197.000000,7.400000,2.057520,1.795715,4.076693
25%,1.300000,0.401600,179.000000,222.000000,223.000000,10.400000,3.370111,2.512846,6.148368
50%,1.400000,0.404800,228.000000,242.000000,242.000000,11.100000,3.701077,2.920633,7.332125
75%,1.600000,0.408200,255.000000,257.000000,255.000000,12.300000,4.229802,3.371688,8.199505
max,2.300000,0.423000,357.000000,305.000000,290.000000,14.000000,5.891803,6.609899,11.542480


In [ ]:
if GENERATE_PLOTS:
    plot_all_columns_group_by_a_categorical_col(list_dataset["std_col_name"], "type")

#### From above plots, we see that "mass", "h_diameter", "v_diameter" and "brm_ta" plots have clearly patterns corresponding with types.

## Analyze qualitative data

In [ ]:
# partitions_dataframe = divide_dataframe_with_categorical_col(list_dataset["std_col_name"], "type")

### Use Welch's t-test to see whether the data could be divided into 3 parts A, B, and C.

In [ ]:
# for group in partitions_dataframe.keys():
#     print(f"{group}: {partitions_dataframe[group].shape}")

In [ ]:
# partitions_dataframe["A"].head(10)

In [ ]:
# columns_to_compare = ["mass", "h_diameter", "v_diameter"]

In [ ]:
# calculate_p_values_between_partitions(partitions_dataframe, columns_to_compare)

### All p-values smaller than $ \alpha $ = 0.05
### => Conclusion from Welch's t-test: The dataset could be divided into 3 parts A, B and C

## Matrix plots

### Pairplots

In [ ]:
if GENERATE_PLOTS:
    plt.figure(figsize=(16, 16))
    sns.pairplot(list_dataset["std_col_name"].get_dataframe())

#### Some potential features to explain brix are "mass", "h_diameter", "v_diameter", "brix_ta", "brm_ta".

### Heatmap

In [ ]:
if GENERATE_PLOTS:
    plt.figure(figsize=(12, 12))
    dataframe = list_dataset["std_col_name"].get_dataframe()
    numeric_cols = list_dataset["std_col_name"].numeric_cols
    dataset_corr = dataframe[numeric_cols].corr()
    sns.heatmap(dataset_corr, annot=True)

#### => Some features that could be used for model: **mass, h_diameter, v_diameter, brix_ta, brm_ta** (quantitative) and **type** (qualitative) to explain **brix**. However, **brix_ta** and **brm_ta** are directly related to **brix**, then they will not be considered.

## Build Models

### Create "ohv_std_col_name" dataframe: ["brix", "mass", "h_diameter", "v_diameter", one-hot-vector-of-type]

In [ ]:
ohv_dataset = list_dataset["std_col_name"].get_one_hot_vectorized_dataset("type")
list_dataset["std_col_name_ohv_type"] = ohv_dataset
list_dataset["std_col_name_ohv_type"].get_dataframe().head(10)

std_col_name_ohv_type


,A,B,C,name,v_naoh,c_naoh,mass,h_diameter,v_diameter,brix,ta,brix_ta,brm_ta,type
0,1,0,0,A1,1.90,0.4000,255,257,262,10.9,4.867141,2.239508,6.032859,A
1,1,0,0,A2,1.20,0.4000,325,288,278,9.8,3.073984,3.188045,6.726016,A
2,1,0,0,A3,2.30,0.4000,185,235,237,11.0,5.891803,1.867001,5.108197,A
3,1,0,0,A4,1.40,0.4128,203,245,245,9.0,3.701077,2.431725,5.298923,A
4,1,0,0,A5,1.20,0.4000,229,253,249,10.9,3.073984,3.545887,7.826016,A
5,1,0,0,A7,1.30,0.4128,357,305,290,10.7,3.436714,3.113439,7.263286,A
6,1,0,0,A8,2.00,0.4000,309,275,285,11.2,5.123307,2.186088,6.076693,A
7,1,0,0,A9,1.90,0.4128,321,285,280,9.2,5.022890,1.831615,4.177110,A
8,1,0,0,A10,1.45,0.4082,245,255,259,9.5,3.790542,2.506238,5.709458,A
9,1,0,0,A11,2.00,0.4128,275,267,270,10.5,5.287252,1.985909,5.212748,A


In [ ]:
y_cols = ["brix"]
y_variable = list_dataset["std_col_name_ohv_type"].get_dataframe()[y_cols]
y_variable.head(10)

,brix
0,10.9
1,9.8
2,11.0
3,9.0
4,10.9
5,10.7
6,11.2
7,9.2
8,9.5
9,10.5


In [ ]:
x_cols = ["mass", "h_diameter", "v_diameter", "B", "C"]
x_variable = list_dataset["std_col_name_ohv_type"].get_dataframe()[x_cols]
x_variable.head(10)

,mass,h_diameter,v_diameter,B,C
0,255,257,262,0,0
1,325,288,278,0,0
2,185,235,237,0,0
3,203,245,245,0,0
4,229,253,249,0,0
5,357,305,290,0,0
6,309,275,285,0,0
7,321,285,280,0,0
8,245,255,259,0,0
9,275,267,270,0,0


### Linear regression models

In [ ]:
model_list =['vit_base_patch16_224.dino']
std_dataset_names = ['std_col_name_ohv_type']

hyper_params = {"SVR": {
          "kernel": ["rbf", "linear", "poly"],
          "C": np.arange(0.1, 2, 0.1)
        },

    "RANDOM_FOREST": {
        "n_estimators": np.arange(100, 1100, 100),
        "max_depth": [None, 5, 10, 15, 20],
        "max_features": [10, 50, 100, 150, 200, "sqrt", "log2", None]
    },
    "LIGHT_GBM": {
        "max_depth": [-1], # , 5, 10, 15, 20, 30],
        "learning_rate":  [0.015], # , 0.05, 0.1],
        "min_child_samples": [20],
        "boosting_type": ['gbdt'],
         "n_estimators": np.arange(1900, 2000, 100),
        # "num_leaves": [15, 20, 31, 40, 50, 64, 70, 75]
    },
    "XGB": {
        "learning_rate": [0.01, 0.1, 0.15,],
        "n_estimators": [1000, 1500, 1900], # int, 100-1000 (step 100)
    }
    }

from sklearn.model_selection import ParameterGrid

param_search = {
    "LIGHT_GBM": list(ParameterGrid(hyper_params["LIGHT_GBM"])),
    # "XGB": list(ParameterGrid(hyper_params["XGB"]))
    }

In [ ]:
model_types = ModelType

In [ ]:
# result_frame: ResultFrame = ResultFrame()

# secondary_model_list = [
#     # "vit_large_patch14_dinov2.lvd142m",
#     # "vit_small_patch14_dinov2.lvd142m",
#     # "eva_large_patch14_196.in22k_ft_in22k_in1k",
#     "vit_base_patch14_dinov2.lvd142m",
#     # 'resnet34'
#     # "vit_base_patch16_224.mae"
# ]
# secondary_options =  [f"get_{k}" for k in [30]]
# options = ["get_10"]
# for model_name in model_list:
#   for option in options:
#     resnet_dataset = pd.read_excel(f"legacy/{model_name}_{option}.xlsx")
#     df_orig = list_dataset["std_col_name_ohv_type"].get_dataframe()
#     for snd_model in secondary_model_list:
#       for opt in secondary_options:
#         df_new = pd.merge(df_orig, resnet_dataset, on = "name")
#         df_snd = pd.read_excel(f"legacy/{snd_model}_{opt}.xlsx")
#         df_new = pd.merge(df_new, df_snd, on = "name")
#         snd_cols = list(df_snd.drop("name", axis = 1).columns)
#         dataset_orig_resnet =  Dataset("orig_resnet", df_new, ScalerType.Raw_)


#         list_dataset["orig_resnet"] = dataset_orig_resnet
#         orig_cols = ['mass', 'h_diameter', 'B', 'C']
#         resnet_cols = list(resnet_dataset.drop("name", axis = 1).columns)

#         x_cols = ['B', 'C'] + resnet_cols
#         feature_all = resnet_cols + snd_cols
#         feature_r2 = {}
#         for feature in resnet_cols + snd_cols:
#           feats_now = [feature]
#           feature_collection = {"orig_cols": orig_cols, "x_cols": x_cols, "resnet_cols": feats_now}
#           for dataset_name in std_dataset_names:
#               for model_type in model_types:
#                 model_family = model_type.__str__().split(".")[-1]
#                 if model_family in ["DECISION_TREE", "XGB", "LINEAR_REGRESSION", "RANDOM_FOREST", "KNN"]:
#                   continue
#                 if model_family in param_search.keys():
#                   for param in param_search[model_family]:
#                     for key, value in param.items():
#                       model_type.value[key] = value
#                     for key, value in feature_collection.items():
#                         if key in ["orig_cols", "x_cols"]:
#                             continue
#                         name = f"{model_family}_{key}_{model_name}_{option}_{str(param)}_{snd_model}_{opt}"
#                         # print(f"Training {name}")
#                         model = Model(
#                             model_type=model_type,
#                             name= name,
#                             x_cols=value,
#                             y_cols=y_cols,
#                             dataset=list_dataset["orig_resnet"],
#                             scaler_type=ScalerType.Raw_,
#                         )
#                         result = model.evaluate()
#                         r2  =    result.result[-2]
#                         mse =   result.result[-1]
#                         feature_r2[feature] = r2
#                         # if r2  > max_R2:
#                         #   max_R2 = r2
#                         #   selected_feats = feats_now
#                         # print(f"R2: {r2} | MSE: {mse}")
#                         result_frame.add_result(result)


# result_frame.display_result()
# print(max_R2)
# print(selected_feats)

# len(feature_r2)
# feature_r2_sorted={k: v for k, v in sorted(feature_r2.items(), key=lambda item: item[1], reverse = True)}
# selected_feats = [feat for feat in feature_r2_sorted.keys() if feature_r2_sorted[feat]>0.0]

1536

['timm/vit_base_patch16_224.dino_743', 'timm/vit_base_patch16_224.dino_212', 'timm/vit_base_patch16_224.dino_478', 'timm/vit_base_patch14_dinov2.lvd142m_733', 'timm/vit_base_patch16_224.dino_295', 'timm/vit_base_patch14_dinov2.lvd142m_393', 'timm/vit_base_patch14_dinov2.lvd142m_283', 'timm/vit_base_patch16_224.dino_131', 'timm/vit_base_patch16_224.dino_472', 'timm/vit_base_patch14_dinov2.lvd142m_19', 'timm/vit_base_patch14_dinov2.lvd142m_224', 'timm/vit_base_patch16_224.dino_563', 'timm/vit_base_patch14_dinov2.lvd142m_66', 'timm/vit_base_patch16_224.dino_353', 'timm/vit_base_patch16_224.dino_98', 'timm/vit_base_patch16_224.dino_367', 'timm/vit_base_patch16_224.dino_162', 'timm/vit_base_patch16_224.dino_763', 'timm/vit_base_patch14_dinov2.lvd142m_622', 'timm/vit_base_patch14_dinov2.lvd142m_667', 'timm/vit_base_patch16_224.dino_669', 'timm/vit_base_patch14_dinov2.lvd142m_365', 'timm/vit_base_patch16_224.dino_517', 'timm/vit_base_patch16_224.dino_217', 'timm/vit_base_patch16_224.dino_735'

In [ ]:
# import json
# with open("selected_feat.json", "w") as f:
#   json.dump(selected_feats, f)

import json
with open("selected_feat.json", "r") as f:
  selected_feats = json.load(f)

In [ ]:
# Chạy leave-one-out CV với bộ tham số tốt nhất
result_frame: ResultFrame = ResultFrame()
secondary_model_list = [
    "vit_base_patch14_dinov2.lvd142m",
]

secondary_options =  [f"get_{k}" for k in [30]]
options = ["get_10"]
for model_name in model_list:
  for option in options:
    resnet_dataset = pd.read_excel(f"legacy/{model_name}_{option}.xlsx")
    df_orig = list_dataset["std_col_name_ohv_type"].get_dataframe()
    for snd_model in secondary_model_list:
      for opt in secondary_options:
        df_new = pd.merge(df_orig, resnet_dataset, on = "name")
        df_snd = pd.read_excel(f"legacy/{snd_model}_{opt}.xlsx")
        df_new = pd.merge(df_new, df_snd, on = "name")
        snd_cols = list(df_snd.drop("name", axis = 1).columns)
        dataset_orig_resnet =  Dataset("orig_resnet", df_new, ScalerType.Raw_)


        list_dataset["orig_resnet"] = dataset_orig_resnet
        orig_cols = ['mass', 'h_diameter', 'C']
        resnet_cols = list(resnet_dataset.drop("name", axis = 1).columns)

        x_cols = ['mass', 'h_diameter', 'C'] + selected_feats
        feature_collection = {"orig_cols": orig_cols, "x_cols": x_cols, "resnet_cols": selected_feats}



        for dataset_name in std_dataset_names:
            for model_type in ModelType:
              model_family = model_type.__str__().split(".")[-1]
              if model_family in ["DECISION_TREE", "XGB", "LINEAR_REGRESSION", "SVR", "RANDOM_FOREST", "KNN"]:
                continue
              for param in param_search[model_family]:
                  for key, value in param.items():
                    model_type.value[key] = value
                  for key, value in feature_collection.items():
                      if key in ["orig_cols", "x_cols"]:
                          continue
                      name = f"{model_family}_{key}_{model_name}_{option}_{str(param)}_{snd_model}_{opt}"
                      print(f"Training {name}")
                      model = Model(
                          model_type=model_type,
                          name= name,
                          x_cols=value,
                          y_cols=y_cols,
                          dataset=list_dataset["orig_resnet"],
                          scaler_type=ScalerType.Raw_,
                      )
                      result = model.evaluate()
                      r2  =    result.result[-2]
                      mse =   result.result[-1]
                      print(f"R2: {r2} | MSE: {mse}")
                      result_frame.add_result(result)


result_frame.display_result()
# result_frame.save_result(f"result_{model_name}_num_leaves_exp.xlsx")

Training LIGHT_GBM_resnet_cols_vit_base_patch16_224.dino_get_10_{'boosting_type': 'gbdt', 'learning_rate': 0.015, 'max_depth': -1, 'min_child_samples': 20, 'n_estimators': 1900}_vit_base_patch14_dinov2.lvd142m_get_30


100%|██████████| 81/81 [02:18<00:00,  1.70s/it]

R2: 0.6086451165084795 | MSE: 0.7177572035968646


,dataset_name,model_name,model_type,scaler_type,r2,mse
0,orig_resnet,LIGHT_GBM_resnet_cols_vit_base_patch16_224.din...,ModelType.LIGHT_GBM,ScalerType.Raw_,0.608645,0.717757
